[原博客地址  --- ](https://kevinjiang.info/2019/01/30/使用Surprise实现电影推荐/)

In [ ]:
ratings_data_path="/Users/zhouwencheng/Desktop/Grass/data/otherdata/103ratings/ratings.csv"

In [1]:
import pandas as pd
from surprise import Reader, Dataset, KNNBaseline

/Users/zhouwencheng/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/zhouwencheng/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [9]:
# 加载数据集

# 评分数据包含四列： userId, movieId, rating, timestamp
ratings = pd.read_csv(ratings_data_path)
# 电影数据包含三列：moveId, title, genres
movies = pd.read_csv('data/movies.csv')

# 把两个csv数据并和成一份数据，以movieId为基准
df = pd.merge(ratings, movies, on='MovieID')
df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [10]:
df.describe()

,UserID,MovieID,Rating,Timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


## 创建训练集

In [37]:
# 需要使用Reader对象来指定输入数据的一些格式
reader = Reader(rating_scale=(0.5, 0.5))
# 通过surprise.Dataset类的load_from_df方法，加载pandas数据格式
# 只支持user, item, rating三列的数据，所以需要从pandas里面提取对应的三列
# 以rader指定的格式来加载数据
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)
# 把加载的数据全部作为训练数据生成训练数据集
trainset = data.build_full_trainset()

## 使用基于用户的算法

In [38]:
# 这里基于用户的算法，所以要指定user_bases为True
# 如果是基于物品的算法，user_base指定为False即可
sim_options = {'name':'pearson_baseline', 'user_based':True}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## 实现推荐算法
- 假设以用户1位指定用户，查找与userid为1的相似用户

In [28]:
# 首先需要把userId转换为surprise内部的id，
# 因为真实的外部userId很有可能是不连续的，不利于计算
inner_user_id = algo.trainset.to_inner_uid(1)
# 通过内部用户id获取邻近（相似）的其他内部用户id
neighbor_users = algo.get_neighbors(inner_user_id, k = 5)
print('neighbor users inner id:', neighbor_users)
# 可能通过to_raw_uid把内部用户id转换为真实的用户id
for u in neighbor_users:
    print('real user id:', algo.trainset.to_raw_uid(u))

neighbor users inner id: [50, 4, 33, 10, 34]
real user id: 3114
real user id: 2355
real user id: 588
real user id: 595
real user id: 1907


In [34]:
df[df.UserID == 1].head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1725,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2250,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
2886,1,3408,4,978300275,Erin Brockovich (2000),Drama
4201,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [36]:
# 查看其他一个相似用户的记录
uid = algo.trainset.to_raw_uid(neighbor_users[0])
df[df.UserID == uid].head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
15100,3114,2918,4,969466947,Ferris Bueller's Day Off (1986),Comedy
20886,3114,2797,3,969467743,Big (1988),Comedy|Fantasy
22306,3114,2321,2,969467396,Pleasantville (1998),Comedy
24530,3114,1270,4,969465661,Back to the Future (1985),Comedy|Sci-Fi
46249,3114,2692,5,969466437,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
